In [1]:
import json
import sys
import asyncio
from pathlib import Path

# Добавляем корневую директорию проекта в пути импорта
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Импортируем необходимые модули
from src.parsers.resume_extractor import ResumeExtractor
from src.parsers.vacancy_extractor import VacancyExtractor
from src.llm_gap_analyzer.llm_gap_analyzer import LLMGapAnalyzer

In [2]:
# Пути к файлам
resume_file_path = Path("tests/fetched_resume_6d807532ff0ed6b79f0039ed1f63386d724a62.json")
vacancy_file_path = Path("tests/fetched_vacancy_120234346.json")

# Загружаем данные
with open(resume_file_path, 'r', encoding='utf-8') as f:
    resume_data = json.load(f)

with open(vacancy_file_path, 'r', encoding='utf-8') as f:
    vacancy_data = json.load(f)

# Парсим данные
resume_parser = ResumeExtractor()
vacancy_parser = VacancyExtractor()

parsed_resume = resume_parser.extract_resume_info(resume_data)
parsed_vacancy = vacancy_parser.extract_vacancy_info(vacancy_data)

# Преобразуем Pydantic модели в словари
parsed_resume_dict = parsed_resume.model_dump()
parsed_vacancy_dict = parsed_vacancy.model_dump()

# Создаем экземпляр LLMGapAnalyzer
gap_analyzer = LLMGapAnalyzer()

# Получаем промпт для gap-анализа (для просмотра)
prompt = gap_analyzer._create_gap_analysis_prompt(parsed_resume_dict, parsed_vacancy_dict)

# Выводим полный промпт
print("=== ПОЛНЫЙ ПРОМПТ ДЛЯ LLM ===")
print(prompt)

=== ПОЛНЫЙ ПРОМПТ ДЛЯ LLM ===

        # Задача: Анализ соответствия резюме требованиям вакансии
        
        Твоя задача - провести детальный анализ соответствия резюме соискателя требованиям вакансии и предоставить конкретные рекомендации по улучшению резюме для повышения шансов на получение данной позиции.
        
        ## Исходные данные
        
        ## РЕЗЮМЕ

### Желаемая должность
LLM Engineer

### Описание навыков
Я - увлеченный специалист, который видит потенциал искусственного интеллекта в упрощении жизни людей и бизнеса. Люблю копаться в LangChain и изучаю как работаю векторные базы данных. Считаю prompt engineering одним из основных навыков для создания грамотных LLM приложений.

### Ключевые навыки
- ChatGPT
- AI
- prompt
- prompt engineering
- Python
- JSON API
- Нейро-сотрудники
- Умение работать в команде
- Искусственный интеллект
- Глубокое обучение
- NLP
- Машинное обучение
- LLM
- AI тренер
- LangChain
- Stable Diffusion
- Чат-бот

### Опыт работы
#### Опы

In [3]:
# Часть 2: Вызываем LLM и получаем результаты gap-анализа

# Определяем асинхронную функцию для вызова gap_analysis
async def run_gap_analysis():
    print("\n=== ЗАПУСК GAP-АНАЛИЗА ===")
    print("Отправляем запрос к LLM API...")
    
    try:
        # Вызываем метод gap_analysis для получения результата
        result = await gap_analyzer.gap_analysis(parsed_resume_dict, parsed_vacancy_dict)
        
        if result:
            print("\n✅ GAP-анализ успешно выполнен!")
            
            # # Выводим количество рекомендаций
            # print(f"Получено {len(result.recommendations)} рекомендаций:")
            
            # # Выводим рекомендации в структурированном виде
            # for i, rec in enumerate(result.recommendations, 1):
            #     print(f"\n{i}. Раздел: {rec.section}")
            #     print(f"   Тип рекомендации: {rec.recommendation_type}")
            #     print("    Детали:")
            #     for j, detail in enumerate(rec.details, 1):
            #         print(f"     {j}. {detail}")
            
            # Выводим полный JSON результата
            print("\n=== ПОЛНЫЙ JSON РЕЗУЛЬТАТА ===")
            print(json.dumps(result.model_dump(), ensure_ascii=False, indent=2))
            
            return result
        else:
            print("\n❌ Ошибка: Не удалось получить результат от LLM")
            return None
    
    except Exception as e:
        print(f"\n❌ Ошибка при выполнении gap-анализа: {e}")
        return None

In [4]:
# Запускаем асинхронную функцию в Jupyter Notebook
result = await run_gap_analysis()  # Для Jupyter нужно использовать top-level await

# Если хотите сохранить результат в файл
if result:
    result_path = Path("gap_analysis_result.json")
    with open(result_path, "w", encoding="utf-8") as f:
        json.dump(result.model_dump(), f, ensure_ascii=False, indent=2)
    print(f"\nРезультат сохранен в файл: {result_path}")


=== ЗАПУСК GAP-АНАЛИЗА ===
Отправляем запрос к LLM API...

✅ GAP-анализ успешно выполнен!

=== ПОЛНЫЙ JSON РЕЗУЛЬТАТА ===
{
  "suggested_resume_title": "LLM Engineer / AI Engineer с опытом работы с LangChain и RAG",
  "suggested_skills_description_for_rewriter": "Добавьте в описание навыков и раздел ключевых навыков чёткие упоминания PyTorch и опыта промышленного развёртывания моделей, а также выделите навыки управления командой и MLOps. Отразите опыт работы с RAG, fine-tuning и методологиями валидации моделей. В описании добавьте упоминания взаимодействия с Agile-командами и сопутствующими ролями (Data Architect, Data Analyst, Data Engineer).",
  "suggested_skill_set_for_rewriter": [
    "PyTorch",
    "NLP",
    "LLM",
    "LangChain",
    "Prompt Engineering",
    "RAG",
    "Fine-tuning моделей",
    "MLOps",
    "Валидация моделей",
    "Управление командой",
    "Аналитические навыки",
    "Коммуникабельность"
  ],
  "experience_reports": [
    {
      "experience_identifier": "